# Bonus Task : Scratch Implementation of ID3 Algorithm

### Importing Necesssary Libraries 

In [1]:
import pandas as pd
import numpy as np 

#### Loading The Dataset

In [3]:
path = "../Datasets/play_tennis.csv"
try :
    initial_train_data = pd.read_csv(path)
    print("File found.")
except FileNotFoundError : 
    print("Couldnt find the file.")


File found.


## A look into Our Dataset

In [4]:
initial_train_data.head()

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


> **As per the observation of the dataset, we know that : <br>
  Features of the Dataset are : outlook , temp , humidity , wind <br>
  Classes : Yes or No <br>
  Class Label : Play**
  

### Calculating The Entropy of The Whole Dataset 

In [32]:
def totalEntropy(data , label , classes):
    noOfClasses = data.shape[0]
    totalentropy = 0
    
    for i in classes : 
        totalClasses = data[data[label] == i].shape[0]
        print("Total Classes is " , str(totalClasses))
        totalClassEntropy = -(totalClasses/noOfClasses) * np.log2(totalClasses / noOfClasses)
        totalentropy += totalClassEntropy  
        
    return totalentropy

Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []
Empty DataFrame
Columns: [outlook, temp, humidity, wind, play]
Index: []


### Computing the Entropy for the Feature

In [34]:
def calculateEntropy(featureValueData , label , classes):
    noOfclasses = featureValueData.shape[0]
    entropy = 0 
    
    for i in classes : 
        '''Row count of the Class'''
        classLabelCount = featureValueData[featureValueData[label] == i].shape[0]
        classEntropy = 0 
        if classLabelCount != 0 :
            ProbabilityOfClass = classLabelCount / noOfclasses
            classEntropy = - ProbabilityOfClass * np.log2(ProbabilityOfClass)
        entropy += classEntropy
        
    return entropy 
            

#### Computing Information Gain For the Features 

In [35]:
def InformationGain(feature , train_data , label , classes):
    featureValues = train_data[feature].unique()
    TotalRow = train_data.shape[0]
    informationContent = 0.0 
    for value in featureValues:
        '''Filtering rows with that feature value'''
        featureValueData = train_data[train_data[feature] == value] 
        featureValueCount = featureValueData.shape[0]
        '''Computing Entropy for that Feature Value'''
        featureValueEntropy = calculateEntropy(featureValueData , label , classes)
        # print(featureValueEntropy)
        featureValueProbability = featureValueCount/ TotalRow
        '''Computing Information Content of the Feature'''
        informationContent += featureValueProbability * featureValueEntropy
       # print(f"Information Gain of {value} is { calculateEntropy(featureValueData , 'play' , classes)}")
        '''Returning the Information Gain'''    
    print(f"The Total Entropy of feature {feature} is {totalEntropy(train_data , label , classes) - informationContent}")
    return totalEntropy(train_data , label , classes) - informationContent 

#### Finding The Feature With Maximum Information Gain

In [36]:

maximumInformationFeature = None

def MaximumInformationGain (train_data , label , classes):
    features = train_data.columns.drop(label)
    maximumInformationGain = 0
    maximumInformationFeature = None
   # print(f"Maximum Information Gain is {maximumInformationGain}")
    for feature in features :
        featureInformationGain = InformationGain(feature , train_data , label , classes)
       # print(f"The Feature Information gain for {feature} is {featureInformationGain}"))
        if  maximumInformationGain < featureInformationGain :
            maximumInformationGain = featureInformationGain
            maximumInformationFeature = feature 
    print(f"----The Maximum Information Feature is {maximumInformationFeature} and gain is {maximumInformationFeature}")
    return maximumInformationFeature        

### Building Tree By Adding Nodes 

In [37]:
def addNode(featureName , train_data , label , classes):
    featureValueCount = train_data[featureName].value_counts(sort= False)
    Tree = {}
    
    for featureValue , count in featureValueCount.iteritems():
       # print(featureValue , count)
        featureValueData = train_data[train_data[featureName] == featureValue]
        
        assignmentCheck = False 
        
        for c in classes : 
            classCount = featureValueData[featureValueData[label] == c].shape[0]
            
            if classCount == count :
                Tree[featureValue] = c 
                train_data = train_data[train_data[featureName] != featureValue]
                assignmentCheck = True 
                
        if not assignmentCheck : 
            Tree[featureValue] = "?"
    return Tree , train_data 

#### Generating Tree With ID3 Algorithm 

In [38]:
def makeTree(root , previousFeatureValue , train_data , label , classes):
    if train_data.shape[0] != 0 :
        rootFeature = MaximumInformationGain (train_data , label , classes)
        Tree , train_data = addNode(rootFeature , train_data , label , classes)
        nextRoot = None 
        
    if previousFeatureValue != None :
        root[previousFeatureValue] = dict()
        root[previousFeatureValue][rootFeature] = Tree 
        nextRoot = root[previousFeatureValue][rootFeature]
    else :
        root[rootFeature] = Tree 
        nextRoot = root[rootFeature]
        
    print(f"Next root {nextRoot}")    
    for node, branch in list(nextRoot.items()):
        print("Node is " , node)
        if branch == "?" : 
            featureValueData = train_data[train_data[rootFeature] == node]
            makeTree(nextRoot , node , featureValueData , label , classes)

### Finding Unique Class Label and Implementing The Algorithm

In [39]:
def ID3(initial_train_data , label):
    train_data = initial_train_data.copy()
    Tree = {}
    classes = train_data[label].unique()
    makeTree(Tree , None , train_data , label , classes)
    return Tree

#### Implementing the Algorithm for the Dataset In Hand 

In [40]:
theTree = ID3(initial_train_data , 'play')

Total Classes is  5
Total Classes is  9
The Total Entropy of feature outlook is 0.24674981977443933
Total Classes is  5
Total Classes is  9
Total Classes is  5
Total Classes is  9
The Total Entropy of feature temp is 0.02922256565895487
Total Classes is  5
Total Classes is  9
Total Classes is  5
Total Classes is  9
The Total Entropy of feature humidity is 0.15183550136234159
Total Classes is  5
Total Classes is  9
Total Classes is  5
Total Classes is  9
The Total Entropy of feature wind is 0.04812703040826949
Total Classes is  5
Total Classes is  9
----The Maximum Information Feature is outlook and gain is outlook
Next root {'Sunny': '?', 'Overcast': 'Yes', 'Rain': '?'}
Node is  Sunny
Total Classes is  3
Total Classes is  2
The Total Entropy of feature outlook is 0.0
Total Classes is  3
Total Classes is  2
Total Classes is  3
Total Classes is  2
The Total Entropy of feature temp is 0.5709505944546686
Total Classes is  3
Total Classes is  2
Total Classes is  3
Total Classes is  2
The To

### The Output Treee is 

In [12]:
from anytree import Node, RenderTree

root = Node('Outlook')

level_1_child_1 = Node('Sunny', parent=root)
level_1_child_2 = Node('Overcast', parent=root)
level_2_child_1 = Node('Humidity', parent=level_1_child_1)
level_3_child_1 = Node('High', parent=level_2_child_1)
level_4_child_1 = Node('No', parent=level_3_child_1)

level_3_child_2 = Node('Normal', parent=level_2_child_1)
level_4_child_2 = Node('Yes', parent=level_3_child_2)
level_2_child_2 = Node('Yes', parent=level_1_child_2)


level_1_child_3 = Node('Rain' , parent = root )
level_2_child_3 = Node('wind' , parent = level_1_child_3  )
level_3_child_3 = Node('Weak' , parent = level_2_child_3  )
level_4_child_3 = Node('Yes' , parent = level_3_child_3  )
level_3_child_4 = Node('Strong' , parent = level_2_child_3  )
level_4_child_4 = Node('No' , parent =  level_3_child_4 )
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))



Outlook
├── Sunny
│   └── Humidity
│       ├── High
│       │   └── No
│       └── Normal
│           └── Yes
├── Overcast
│   └── Yes
└── Rain
    └── wind
        ├── Weak
        │   └── Yes
        └── Strong
            └── No
